In [188]:
import os
from timeit import default_timer as timer
from glob import glob
import pandas as pd
import numpy as np

In [252]:
country_code = "US"
if os.getenv('CLUSTER')=='PRINCE':
    path_to_data='/scratch/spf248/twitter/data'
else:
    path_to_data='../../data'

In [297]:
if country_code=='US':
    user_location_2_geo_id=pd.read_csv(os.path.join(path_to_data,'official','city',country_code,'user_location_2_geo_id_admin_2.csv'),index_col=0)
    # Create Geo Id of Metro Areas
    metro_area_name_2_geo_id=user_location_2_geo_id.drop_duplicates(
    'metro_area_name').reset_index(drop=True).reset_index(
    ).set_index('metro_area_name')['index'].rename('geo_id')
    user_location_2_geo_id['geo_id']=user_location_2_geo_id['metro_area_name'].apply(
    lambda x:metro_area_name_2_geo_id[x])
    user_location_2_geo_id.to_csv(os.path.join(path_to_data,'official','city',country_code,'user_location_2_geo_id.csv'))

In [235]:
if country_code!='US':
    labor_market_stats=pd.read_csv(os.path.join(path_to_data,'official','city',country_code,'labor_market_stats.csv'),sep={'AR':';','BR':',','CO':';','MX':','}[country_code])
    labor_market_stats.rename(columns={'value':'unemployment_rate','NAME':'metro_area_name'},inplace=True)
    labor_market_stats['date']=pd.to_datetime(labor_market_stats.apply(lambda x:str(x['year'])+'-Q'+str(x['quarter']),1)) + pd.offsets.QuarterEnd(0)
    labor_market_stats['month'] = labor_market_stats['date'].apply(lambda x:x.month)
    labor_market_stats = labor_market_stats[['year','month','geo_id','unemployment_rate']].sort_values(
    by=['year','month','geo_id']).reset_index(drop=True)
    if country_code =='CO':
        labor_market_stats.unemployment_rate=labor_market_stats.unemployment_rate.apply(lambda x:np.float(x.replace(',','.'))/100)
    labor_market_stats.to_csv(os.path.join(path_to_data,'official','city',country_code,'time_series_unemployment_rate.csv'))

# US - compute average rate across counties

In [330]:
if country_code=='US':
    labor_market_stats=pd.read_csv(os.path.join(path_to_data,'official','city',country_code,'labor_market_stats.csv'),index_col=0)
    pop_metro_area=pd.read_csv(os.path.join(path_to_data,'official','city',country_code,'pop_metro_area.csv'),index_col=0)
    pop_metro_area=pop_metro_area.groupby(['metro_area_name','geo_id'],as_index=False)['pop'].mean()
    df=labor_market_stats.merge(pop_metro_area,on='geo_id')
    df['unemployment_rate']=df['unemployment_rate'].multiply(df['pop'])
    df=df.groupby(['year','month','metro_area_name']).sum()
    df=df['unemployment_rate'].divide(df['pop']).rename('unemployment_rate')
    df/=100
    df=df.to_frame().reset_index()
    df['geo_id']=df['metro_area_name'].apply(lambda x:metro_area_name_2_geo_id.get(x,np.nan))
    df=df.sort_values(by=['year','month','geo_id']).reset_index(drop=True).drop(['metro_area_name'],1).dropna()
    df['geo_id']=df['geo_id'].astype(int)
    df=df[['year', 'month', 'geo_id', 'unemployment_rate']].copy()
    df.to_csv(os.path.join(path_to_data,'official','city',country_code,'time_series_unemployment_rate.csv'))

/Users/samuelfraiberger/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
